# 연관분석 개요

In [ ]:
# 유사성 : 이것과 이것 같고
#연관석 분석 : A가 나오면, B가 나온다 

<pre>
- 데이터들 사이에서 '자주 발생하는 속성을 찾고, 그 속성들 사이에 어느 정도 있는지를 분석하는 방법 
- 활용분야: '상품진열', '사기보험적발', '카탈로그디자인', '신상품 카테고리 구성' 등 
- 연관성분석 관련 지표: 
    지지도(support)     =     조건결과항목수 / 전체수 
    신뢰도 (confidence) =     조건결과항목수 / 조건항목수
    향상도(lift)        :     우연히 발생한 규칙인지 아닌지 여부 
                              = 조건결과지지도 / (조건지지도)*(결과지지도)
                              1: 상관관계없음
                              >1: 양의 상관관계  
                              <1: 음의 상관관계 (유의미한 관계 아님)
    
    [조건] => [결과]        지지도              신뢰도                향상도 
    [주스] => [콜라]     0.4 (2나누기5)          1 (2나누기2)         0.4/(0.4)*1   (콜라의 '결과지지'도 5번중에 5번 다나옴.so 1)
    [소주] => [맥주]     0.2             0.333 (소주를산3명의 사람중에 1명만 소주&맥주삼)      0.2(1나누기5) / (0.6*0.4) = 0.8333


In [1]:
0.2/ (0.6*0.4)

0.8333333333333334

In [2]:
import os
os.getcwd()

'D:\\bigdata(psa)\\src\\9_자연어 처리'

In [8]:
# 트렌젝션 데이터 가져오기 
import csv
transaction = []
with open('cf_basket.csv','r',encoding='utf-8') as cf:
    csvdata = csv.reader(cf)
    for row in csvdata:
        transaction.append(row)
    #transaction = list(csvdata)


In [10]:
transaction

[['소주', '콜라', '와인'],
 ['소주', '오렌지주스', '콜라'],
 ['콜라', '맥주', '와인'],
 ['소주', '콜라', '맥주'],
 ['오렌지주스', '와인', '콜라']]

# 연관분석 
- pip install apyori

In [13]:
from apyori import apriori
rules = apriori(transaction, min_support=0.2, min_confidence=0.1)
rules = list(rules)
len(rules)


19

In [15]:
rules[1]

RelationRecord(items=frozenset({'소주'}), support=0.6, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'소주'}), confidence=0.6, lift=1.0)])

In [20]:
print('조건 -> 결과 \t 지지도 \t 신뢰도 \t 향상도')
for row in rules:
    support = row[1]
    ordered_statistics= row[2] #ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'소주'}), confidence=0.6, lift=1.0)])
    for item in ordered_statistics: 
        #lhs = ','.join(x for x in item[0])
        lhs = [ x for x in item[0] ]  #list형태로 하나만 들어 있을 수 있다        #items_add=
        rhs = ','.join(x for x in item[1])              #rhs   right hand side   #frozenset({'소주'}
        confidence = item[2]
        lift= item[3]
        if lift != 1:
            print(lhs,"=>", rhs,"\t{:5.3f}\t{:5.3f}\t{:5.3f}".format(support,confidence,lift) )

            
            

조건 -> 결과 	 지지도 	 신뢰도 	 향상도
['맥주'] => 소주 	0.200	0.500	0.833
['소주'] => 맥주 	0.200	0.333	0.833
['맥주'] => 와인 	0.200	0.500	0.833
['와인'] => 맥주 	0.200	0.333	0.833
['소주'] => 오렌지주스 	0.200	0.333	0.833
['오렌지주스'] => 소주 	0.200	0.500	0.833
['소주'] => 와인 	0.200	0.333	0.556
['와인'] => 소주 	0.200	0.333	0.556
['오렌지주스'] => 와인 	0.200	0.500	0.833
['와인'] => 오렌지주스 	0.200	0.333	0.833
['맥주'] => 콜라,소주 	0.200	0.500	0.833
['소주'] => 맥주,콜라 	0.200	0.333	0.833
['맥주', '콜라'] => 소주 	0.200	0.500	0.833
['콜라', '소주'] => 맥주 	0.200	0.333	0.833
['맥주'] => 와인,콜라 	0.200	0.500	0.833
['와인'] => 맥주,콜라 	0.200	0.333	0.833
['맥주', '콜라'] => 와인 	0.200	0.500	0.833
['와인', '콜라'] => 맥주 	0.200	0.333	0.833
['소주'] => 오렌지주스,콜라 	0.200	0.333	0.833
['오렌지주스'] => 콜라,소주 	0.200	0.500	0.833
['콜라', '소주'] => 오렌지주스 	0.200	0.333	0.833
['오렌지주스', '콜라'] => 소주 	0.200	0.500	0.833
['소주'] => 와인,콜라 	0.200	0.333	0.556
['와인'] => 콜라,소주 	0.200	0.333	0.556
['콜라', '소주'] => 와인 	0.200	0.333	0.556
['와인', '콜라'] => 소주 	0.200	0.333	0.556
['오렌지주스'] => 와인,콜라 	0.200	0.500	0.833
['와인'] 

# 뉴스기사 연관 분석 실습

## 뉴스 RSS 이용해서 기사 검색 후 연관분석 
- https://rss.joins.com/joins_money_list.xml

In [25]:
import requests 
from bs4 import BeautifulSoup
rss_url = "https://rss.joins.com/joins_money_list.xml"
money_response = requests.get(rss_url)
money_soup = BeautifulSoup(money_response.content,"xml")
link_list = money_soup.select("item > link")
link_list = [l.text for l in link_list]  #******링크주소만 나오게**************
link_list

['https://news.joins.com/article/24021331?cloc=rss-news-economy',
 'https://news.joins.com/article/24021323?cloc=rss-news-economy',
 'https://news.joins.com/article/24021317?cloc=rss-news-economy',
 'https://news.joins.com/article/24021309?cloc=rss-news-economy',
 'https://news.joins.com/article/24021300?cloc=rss-news-economy',
 'https://news.joins.com/article/24021294?cloc=rss-news-economy',
 'https://news.joins.com/article/24021290?cloc=rss-news-economy',
 'https://news.joins.com/article/24021287?cloc=rss-news-economy',
 'https://news.joins.com/article/24021263?cloc=rss-news-economy',
 'https://news.joins.com/article/24021234?cloc=rss-news-economy',
 'https://news.joins.com/article/24021228?cloc=rss-news-economy',
 'https://news.joins.com/article/24021164?cloc=rss-news-economy',
 'https://news.joins.com/article/24021110?cloc=rss-news-economy',
 'https://news.joins.com/article/24021095?cloc=rss-news-economy',
 'https://news.joins.com/article/24021080?cloc=rss-news-economy',
 'https://

In [32]:
from konlpy.tag import Kkma
kkma = Kkma()
news = []
for link in link_list:
    news_response = requests.get(link)   #위에는 list이기때문에 .text붙이면 안됨 
    news_soup = BeautifulSoup(news_response.content,"html.parser")
    content = news_soup.select_one('div#article_body').text
    nouns_list = list(filter(lambda word: len(word)>1, kkma.nouns(content)))
    news.append(nouns_list)
news

[['26',
  '26일',
  '승진',
  '대상',
  '대상그룹',
  '그룹',
  '부회장',
  '회장',
  '사진',
  '44',
  '전무',
  '지주',
  '지주회사',
  '회사',
  '대상홀딩스와',
  '홀딩',
  '스와',
  '임창',
  '명예',
  '명예회장',
  '장녀',
  '차녀',
  '임상',
  '임상민',
  '41',
  '자매',
  '경영',
  '체제',
  '본격',
  '궤도',
  '해석',
  '부회',
  '대상홀딩스',
  '딩스',
  '전략',
  '전략담당',
  '담당',
  '중역',
  '기존',
  '마케팅',
  '마케팅담당',
  '보직',
  '동시',
  '수행',
  '관련',
  '대상홀딩',
  '이날',
  '정기',
  '주주',
  '주주총회',
  '총회',
  '사내',
  '사내이사',
  '이사',
  '신규',
  '선임',
  '안건',
  '의결',
  '연세대',
  '경영학',
  '공부',
  '이후',
  '미국',
  '뉴욕',
  '뉴욕대',
  '심리학',
  '전공',
  '1998',
  '1998년',
  '이재',
  '삼성',
  '삼성전자',
  '전자',
  '결혼',
  '2009',
  '2009년',
  '이혼',
  '2012',
  '2012년',
  '크리',
  '티브',
  '디렉터',
  '직책',
  '합류',
  '2014',
  '2014년',
  '청정원',
  '정원',
  '대규모',
  '리뉴얼',
  '주도',
  '주도해',
  '브랜드',
  '이미지',
  '강화',
  '2016',
  '2016년',
  '안주야',
  '주야',
  '출시',
  '국내',
  '안주',
  '가정',
  '가정간편식',
  '편식',
  '시장',
  '개척',
  '설명',
  '관계자',
  '변화',
  '정확',
  '정보',
  '습득',
  '실행',
  '차원',
  '중장기',


In [33]:
from apyori import apriori
rules = apriori(news, min_support=0.3, min_confidence=0.2)
result = list(rules)

In [34]:
for r in result:
    print(r,end="\n\n")

RelationRecord(items=frozenset({'10'}), support=0.3333333333333333, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'10'}), confidence=0.3333333333333333, lift=1.0)])

RelationRecord(items=frozenset({'19'}), support=0.3333333333333333, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'19'}), confidence=0.3333333333333333, lift=1.0)])

RelationRecord(items=frozenset({'26'}), support=0.6, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'26'}), confidence=0.6, lift=1.0)])

RelationRecord(items=frozenset({'26일'}), support=0.6, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'26일'}), confidence=0.6, lift=1.0)])

RelationRecord(items=frozenset({'결과'}), support=0.3, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'결과'}), confidence=0.3, lift=1.0)])

RelationRecord(items=frozenset({'계획'}), support=0.36666666666666664, ordered_s

In [36]:
result[3]

RelationRecord(items=frozenset({'26일'}), support=0.6, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'26일'}), confidence=0.6, lift=1.0)])

In [41]:
import pandas as pd
result_df = pd.DataFrame(None, 
                        columns=['lhs','rhs','support','confidence','lift'])
index = 0
for row in result:
    support = row[1]
    ordered_statistics = row[2]
    for item in ordered_statistics:
        lhs = ','.join(x for x in item[0])
        rhs = ','.join(x.strip()for x in item[1])
        confidence = item[2]
        lift = item[3]
        if list !=1:
            result_df.loc[index]=[lhs,rhs,support,confidence,lift]
            index +=1
result_df

,lhs,rhs,support,confidence,lift
0,,10,0.333333,0.333333,1.000000
1,,19,0.333333,0.333333,1.000000
2,,26,0.600000,0.600000,1.000000
3,,26일,0.600000,0.600000,1.000000
4,,결과,0.300000,0.300000,1.000000
...,...,...,...,...,...
913,"주주,주주총회,이사,그룹",총회,0.300000,1.000000,3.333333
914,"주주,총회,이사,그룹",주주총회,0.300000,1.000000,3.333333
915,"총회,주주총회,이사,그룹",주주,0.300000,1.000000,3.000000
916,"주주,주주총회,총회,그룹",이사,0.300000,1.000000,3.333333


### 전체 다 보기 pd.options.display.max_rows = 

In [ ]:
pd.options.display.max_rows = 918
result_df

In [ ]:
#lhs에 '기자' 나오기

In [45]:
result_df.loc[result_df.lhs.str.contains("기자")]. sort_values(by=['lift'],
                                                            ascending=False)  
#step1:  result_df.lhs.str.contains("기자")  step2: 밖에 result_df.loc   step3:  sort_values

,lhs,rhs,support,confidence,lift
845,"관련,기자,26","기사,26일",0.333333,0.769231,2.307692
846,"기사,기자,26","관련,26일",0.333333,1.000000,2.307692
848,"관련,기자,26일","기사,26",0.333333,0.769231,2.307692
849,"기사,기자,26일","관련,26",0.333333,1.000000,2.307692
700,"기사,기자","관련,26",0.333333,0.833333,1.923077
730,"기사,기자","관련,26일",0.333333,0.833333,1.923077
840,"기사,기자","관련,26일,26",0.333333,0.833333,1.923077
880,"이날,기자,26일","관련,26",0.300000,0.818182,1.888112
878,"관련,기자,26일","이날,26",0.300000,0.692308,1.888112
877,"이날,기자,26","관련,26일",0.300000,0.818182,1.888112


In [46]:
#결과 (rhs)에 '기자' 나오게
result_df.loc[result_df.rhs.str.contains("기자")]. sort_values(by=['lift'],
                                                            ascending=False)  

,lhs,rhs,support,confidence,lift
835,"관련,26일","기사,기자,26",0.333333,0.769231,2.307692
836,"기사,26일","관련,기자,26",0.333333,1.000000,2.307692
833,"기사,26","관련,기자,26일",0.333333,1.000000,2.307692
832,"관련,26","기사,기자,26일",0.333333,0.769231,2.307692
695,"관련,26","기사,기자",0.333333,0.769231,1.923077
841,"관련,26일,26","기사,기자",0.333333,0.769231,1.923077
725,"관련,26일","기사,기자",0.333333,0.769231,1.923077
868,"이날,26일","관련,기자,26",0.300000,0.818182,1.888112
866,"관련,26일","이날,기자,26",0.300000,0.692308,1.888112
865,"이날,26","관련,기자,26일",0.300000,0.818182,1.888112


In [47]:
result_df.loc[(result_df.lhs.str.contains("기자"))&  result_df.rhs.str.contains("한국") ]. sort_values(by=['lift'],
                                                            ascending=False)  

,lhs,rhs,support,confidence,lift
482,"관련,기자",한국,0.333333,0.555556,1.111111
480,기자,"관련,한국",0.333333,0.434783,1.086957
218,기자,한국,0.366667,0.478261,0.956522


In [ ]:
test_url="https://search.daum.net/search?w=tot&DA=YZR&t__nil_searchbox=btn&sug=&sugo=&sq=&o=&q=%EC%BD%94%EB%A1%9C%EB%82%98"

In [ ]:
#셀레니옴. 동적 크롤링 